In [2]:
import sys, os, boto3
import rasterio

import geopandas as gpd
import pandas as pd

from tqdm.notebook import tqdm
from shapely.geometry import box

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [8]:
in_file_list = '/home/wb411133/temp/fathom_files.csv'

s3_bucket = 'wbg-geography01'
s3_prefix = 'FATHOM/v2023/'
s3_folder = 'GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0'

in_files = pd.read_csv(in_file_list)
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(s3_bucket)

In [40]:
# Read and process from disk
all_extents = []
in_folder = '/home/wb411133/temp/FATHOM/FATHOM_TESTING'
for tile_file in tqdm(os.listdir(in_folder)):
    inR = rasterio.open(os.path.join(in_folder, tile_file))
    shp = box(*inR.bounds)
    all_extents.append([tile_file.replace(".tif", ""), shp])

  0%|          | 0/20624 [00:00<?, ?it/s]

In [41]:
fluvial_tiles = gpd.GeoDataFrame(pd.DataFrame(all_extents, columns=['ID', 'geometry']), geometry='geometry', crs=4326)

In [50]:
# Add a column to extents file indicating if a coastal cell is present
cur_out_folder = os.path.join(s3_prefix, 'GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0')
# Get a list of all tiles in a coastal folder
all_files = []
for obj in tqdm(my_bucket.objects.filter(Prefix=cur_out_folder)):
    all_files.append(os.path.basename(obj.key).replace(".tif", ""))

# Add a boolean flag in the fluvial catalog if there is a corresponding coastal tile
fluvial_tiles['COASTAL'] = 0
for idx, row in tqdm(xx.iterrows()):
    if row['ID'] in all_files:
        fluvial_tiles.loc[idx, 'COASTAL'] = 1
        
# There are ~300 coastal tiles that don't have a corresponding fluvial tile
coastal_missing = [x for x in all_files if not x in xx['ID'].values]
len(coastal_missing)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

369

In [51]:
# USE AWS CLI to download the missing tiles 
for c_tile in coastal_missing:
    aws_command = f'aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/{c_tile}.tif .'
    print(aws_command)
    print(' ')

aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n05w088.tif .
 
aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n06w163.tif .
 
aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n10w110.tif .
 
aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n15e146.tif .
 
aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n29e140.tif .
 
aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n31e139.tif .
 
aws s3 cp s3://wbg-geography01/FATHOM/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/n50e144.tif .
 
aws s3 cp s3://wbg-geograph

In [52]:
# Loop through downloaded files to get extents
missing_tiles_folder = '/home/wb411133/temp/FATHOM/COASTAL_MISSING'
all_extents = []

for tile_file in tqdm(os.listdir(missing_tiles_folder)):
    inR = rasterio.open(os.path.join(missing_tiles_folder, tile_file))
    shp = box(*inR.bounds)
    all_extents.append([tile_file.replace(".tif", ""), shp])
coastal_missing_extents = gpd.GeoDataFrame(pd.DataFrame(all_extents, columns=['ID', 'geometry']), geometry='geometry', crs=4326)

  0%|          | 0/357 [00:00<?, ?it/s]

In [54]:
# Add corresponding flags to final version
fluvial_tiles['FLUVIAL'] = 1
coastal_missing_extents['FLUVIAL'] = 0
coastal_missing_extents['COASTAL'] = 1
combo_extents = fluvial_tiles.append(coastal_missing_extents)

In [56]:
combo_extents.head()

,ID,geometry,FLUVIAL,COASTAL
0,n00e135,"POLYGON ((135.99986 0.00014, 135.99986 1.00014...",1,0
1,n00w065,"POLYGON ((-64.00014 0.00014, -64.00014 1.00014...",1,0
2,n00e007,"POLYGON ((7.99986 0.00014, 7.99986 1.00014, 6....",1,0
3,n00e006,"POLYGON ((6.99986 0.00014, 6.99986 1.00014, 5....",1,1
4,n00w082,"POLYGON ((-81.00014 0.00014, -81.00014 1.00014...",1,0


In [57]:
combo_extents.tail()

,ID,geometry,FLUVIAL,COASTAL
352,s57w028,"POLYGON ((-27.00014 -56.99986, -27.00014 -55.9...",0,1
353,s58w027,"POLYGON ((-26.00014 -57.99986, -26.00014 -56.9...",0,1
354,s59w027,"POLYGON ((-26.00014 -58.99986, -26.00014 -57.9...",0,1
355,s60w027,"POLYGON ((-26.00014 -59.99986, -26.00014 -58.9...",0,1
356,s60w028,"POLYGON ((-27.00014 -59.99986, -27.00014 -58.9...",0,1


In [58]:
combo_extents.to_file('/home/wb411133/temp/fathom_tile_extents.geojson', driver='GeoJSON')